In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Introduction

## Hierarchical Models

In [ ]:
from bayes_tutorial.data import load_ice_cream

In [ ]:
data = load_ice_cream()
data.head()

## Logit transforms

## Hierarchical model for "indie" stores

In [ ]:
import pymc3 as pm

n_shops = len(data["owner_idx"].unique())
with pm.Model() as model:
    logit_p_population = pm.Normal("logit_p_population", mu=0, sigma=1)
    logit_p_owner = pm.Normal("logit_p_owner", mu=logit_p_population, sigma=1, shape=(n_shops,))       # somewhat uninformative priors
    
    logit_p_shop = pm.Normal("logit_p_shop", mu=logit_p_owner[data["owner_idx"]], sigma=3, shape=(len(data),))
    
    p_shop = pm.Deterministic("p_shop", pm.invlogit(logit_p_shop))
    p_owner = pm.Deterministic("p_owner", pm.invlogit(logit_p_owner))
    like = pm.Binomial("like", n=data["num_customers"], p=p_shop, observed=data["num_favs"])
    

In [ ]:
with model:
    trace = pm.sample(2000)

In [ ]:
import arviz as az

with model:
    trace = az.from_pymc3(trace, coords={"p_shop_dim_0": data["shopname"]})

In [ ]:
az.plot_posterior(trace, var_names=["p_owner"]);

In [ ]:
az.plot_forest(trace, var_names=["p_owner"]);

In [ ]:
import janitor

quantiles = (
    trace.posterior
    .stack(draws=("chain", "draw"))
    ["p_owner"]
    .quantile(q=[0.03, 0.5, 0.97], dim=("draws"))
)
quantiles = (
    quantiles
    .to_dataframe()
    .reset_index()
    .pivot_table(columns="quantile", index="p_owner_dim_0", values="p_owner")
    .join_apply(lambda x: x[0.97] - x[0.03], "width")
)
quantiles